In [1]:
#LOADS LIBRARIES
import os
import sys
import warnings
warnings.simplefilter('ignore')
import pickle
#import dill as pickle

import numpy as np
import pandas as pd
import math

import time 
import datetime
from dateutil.relativedelta import relativedelta

In [3]:
os.getcwd()

'C:\\Users\\willi\\Documents\\EJERCICIOS\\OPI'

In [36]:
#Defines directories

INPUT_DIR = r'C:\Users\willi\Documents\EJERCICIOS\OPI\ '.strip()

OUTPUT_DIR = r'C:\Users\willi\Documents\EJERCICIOS\OPI\outputs'.strip()

# FIXES AND FORMATING COLUMNS

In [10]:
#Imports product catalogue

FILE_NAME = '48fcb848-220c-4af0-839b-4fd8ac812c0f.csv'

df_invest = pd.read_csv(INPUT_DIR + FILE_NAME, encoding='latin-1',sep=',',dtype='str')

print(df_invest.shape)
print('\n')

i = 0
for c in df_invest.columns.tolist():
    i = i +1
    print(i, '-', c)

(1168250, 20)


1 - _id
2 - ao_hechos
3 - mes_hechos
4 - fecha_hechos
5 - ao_inicio
6 - mes_inicio
7 - fecha_inicio
8 - delito
9 - fiscalia
10 - agencia
11 - unidad_investigacion
12 - categoria_delito
13 - calle_hechos
14 - calle_hechos2
15 - colonia_hechos
16 - alcaldia_hechos
17 - competencia
18 - longitud
19 - latitud
20 - tempo


In [11]:
df_raw = df_invest.copy()

In [12]:
#ADJUSTS VARIABLE NAMES

df_input = df_raw.copy()

D_COLS_01 = df_input.dtypes.reset_index()['index']

D_COLS =[]
D_VARS = {}
for x in D_COLS_01:
    D_NAME = x.strip().lower()
    D_NAME = D_NAME.replace(" ", '_')
    D_NAME = D_NAME.replace(".", '')
    D_NAME = D_NAME.replace("�", '')
    D_NAME = D_NAME.replace("-", '_')
    D_NAME = D_NAME.replace("/", '_')
    
    ls_dupnames = [x for x in D_COLS if D_NAME in x]
    if len(ls_dupnames)>0:
        D_NAME = D_NAME + '_' + str(len(ls_dupnames))
        
    
    d = {D_NAME:str}
    D_VARS.update(d)
    
    D_COLS.append(D_NAME)


df_input.columns = D_VARS

df_raw_01 = df_input.copy()

i = 0
for c in df_raw_01.columns.tolist():
    i = i +1
    print(i, '-', c)

1 - _id
2 - ao_hechos
3 - mes_hechos
4 - fecha_hechos
5 - ao_inicio
6 - mes_inicio
7 - fecha_inicio
8 - delito
9 - fiscalia
10 - agencia
11 - unidad_investigacion
12 - categoria_delito
13 - calle_hechos
14 - calle_hechos2
15 - colonia_hechos
16 - alcaldia_hechos
17 - competencia
18 - longitud
19 - latitud
20 - tempo


In [13]:
df_informats = df_raw_01.dtypes.reset_index()
df_informats = df_informats[df_informats[0]!='object']

df_informats

,index,0


In [16]:
print(df_raw_01['ao_hechos'].value_counts(dropna=False).reset_index().sort_values(['index']))
print(df_raw_01['ao_inicio'].value_counts(dropna=False).reset_index().sort_values(['index']))

   index  ao_hechos
70  1906          1
58  1915          2
67  1930          1
68  1942          1
53  1950          2
..   ...        ...
0   2018     253341
1   2019     242484
3   2020     195646
5   2021      26334
14   NaN        433

[72 rows x 2 columns]
  index  ao_inicio
4  2016     197913
2  2017     228323
0  2018     255311
1  2019     250386
3  2020     204036
5  2021      32281


In [17]:
#SUGESTS WICH VARIABLES COULD BE NUMERICAL AND WICH ONES CATEGORICAL
#this is just a suggestion, it should be commited into a visual inspection, to put the corrects ones

df_input = df_raw_01.copy()

TT_COLS = df_input.columns.tolist()

LS_SUGNUMVAR=[]
LS_SUGCATVAR=[]
LS_SUGVAREXC=[]

for c in TT_COLS:
    DF_VARCOUNTS = df_input[c].value_counts(dropna=False).reset_index().sort_values(['index'])
    PP_CATEGORIES = DF_VARCOUNTS.shape[0]/df_input.shape[0]
    
    if (PP_CATEGORIES > 0.05) | (DF_VARCOUNTS.shape[0] > 1000):
        LS_SUGNUMVAR.append(c)
    else:
        if DF_VARCOUNTS.shape[0] <= 1:
            LS_SUGVAREXC.append(c)
        else:
            LS_SUGCATVAR.append(c)

LS_SUGNUMVAR = [x for x in LS_SUGNUMVAR if x not in df_informats['index'].tolist()]
LS_SUGCATVAR = [x for x in LS_SUGCATVAR if x not in df_informats['index'].tolist()]    
LS_SUGVAREXC = [x for x in LS_SUGVAREXC if x not in df_informats['index'].tolist()]

print(r'Number of Suggested Numerical Variables: ' + str(len(LS_SUGNUMVAR)))
print(r'Number of Suggested Categorical Variables: ' + str(len(LS_SUGCATVAR)))
print(r'Number of Suggested Variables to Exclude: ' + str(len(LS_SUGVAREXC)))


Number of Suggested Numerical Variables: 8
Number of Suggested Categorical Variables: 11
Number of Suggested Variables to Exclude: 1


In [18]:
#PRINTS THE COUNTS FOR THE SUGGESTED NUMERICAL VARIABLES
#this prints helps to do the visual inspection
df_input = df_raw_01.copy()

for c in LS_SUGNUMVAR:
    print(df_input[c].value_counts(dropna=False).reset_index().sort_values(['index']))

          index  _id
904585        1    1
272480       10    1
348500      100    1
348152     1000    1
512076    10000    1
...         ...  ...
448605   999995    1
561469   999996    1
671114   999997    1
1087386  999998    1
1034962  999999    1

[1168250 rows x 2 columns]
                      index  fecha_hechos
372174  1906-06-02 19:00:00             1
368852  1915-01-10 15:00:00             1
197866  1915-09-10 10:00:00             1
242770  1930-02-16 12:00:00             1
376950  1942-10-22 19:00:00             1
...                     ...           ...
243092  2021-02-28 20:14:00             1
342423  2021-02-28 20:20:00             1
225347  2021-02-28 20:30:00             1
331899  2021-02-28 23:00:00             1
0                       NaN           433

[387419 rows x 2 columns]
                      index  fecha_inicio
923598  2016-01-01T01:35:06             1
835587  2016-01-01T01:44:49             1
161007  2016-01-01T02:09:12             1
629019  2016-01-01T02

In [19]:
#NUMERICAL VARIABLES DEFINED AS CATEGORICAL AFTER VISUAL INSPECTION IN THE PREVIOUS STEP

LS_NUMVAREX = [
    'fecha_hechos',
    'fecha_inicio',
    'calle_hechos',
    'calle_hechos2',
    'colonia_hechos',
]



#DEFINES THE LIST OF NUMERICAL VARIABLES
LS_NUMVAR_01 = [x for x in LS_SUGNUMVAR if x not in LS_NUMVAREX]
LS_NUMVAR_01

['_id', 'longitud', 'latitud']

In [20]:
#PRINTS THE COUNTS FOR THE SUGGESTED CATEGORICAL VARIABLES
#this prints helps to do the visual inspection
df_input = df_raw_01.copy()

for c in LS_SUGCATVAR:
    print(df_input[c].value_counts(dropna=False).reset_index().sort_values(['index']))

   index  ao_hechos
70  1906          1
58  1915          2
67  1930          1
68  1942          1
53  1950          2
..   ...        ...
0   2018     253341
1   2019     242484
3   2020     195646
5   2021      26334
14   NaN        433

[72 rows x 2 columns]
         index  mes_hechos
11       Abril       88039
4       Agosto       98173
9    Diciembre       93205
0        Enero      108982
1      Febrero      105723
8        Julio       93439
10       Junio       91482
3        Marzo       99028
7         Mayo       94518
5    Noviembre       96682
2      Octubre      102462
6   Septiembre       96084
12         NaN         433
  index  ao_inicio
4  2016     197913
2  2017     228323
0  2018     255311
1  2019     250386
3  2020     204036
5  2021      32281
         index  mes_inicio
11       Abril       88170
4       Agosto       98721
8    Diciembre       92344
1        Enero      108088
0      Febrero      108490
7        Julio       94483
10       Junio       91212
3        M

In [21]:
#CATEGORICAL VARIABLES DEFINES AS NUMERICAL AFTER VISUAL INSPECTION IN THE PREVIOUS STEP

LS_CATVAREX = [
    ]


#DEFINES THE LIST OF CATEGORICAL VARIABLES
LS_CATVAR_01 = [x for x in LS_SUGCATVAR if x not in LS_CATVAREX]
LS_CATVAR_01

['ao_hechos',
 'mes_hechos',
 'ao_inicio',
 'mes_inicio',
 'delito',
 'fiscalia',
 'agencia',
 'unidad_investigacion',
 'categoria_delito',
 'alcaldia_hechos',
 'competencia']

In [22]:
#PRINTS THE COUNTS FOR THE SUGGESTED VARIABLES TO EXCLUDE
#this prints helps to do the visual inspection
df_input = df_raw_01.copy()

for c in LS_SUGVAREXC:
    print(df_input[c].value_counts(dropna=False).reset_index().sort_values(['index']))

   index    tempo
0    NaN  1168250


In [23]:
#VARIABLES TO KEEP FROM THE EXCLUDE SUGGESTION

LS_SUGVARINC = [
    ]

#DEFINES THE LIST OF CATEGORICAL VARIABLES
LS_SUGVAREXC = [x for x in LS_SUGVAREXC if x not in LS_SUGVARINC]
LS_SUGVAREXC

['tempo']

In [24]:
#FINAL LIST OF CATEGORICAL VARIABLES
LS_CATVAR = LS_CATVAR_01 + LS_NUMVAREX
LS_CATVAR

['ao_hechos',
 'mes_hechos',
 'ao_inicio',
 'mes_inicio',
 'delito',
 'fiscalia',
 'agencia',
 'unidad_investigacion',
 'categoria_delito',
 'alcaldia_hechos',
 'competencia',
 'fecha_hechos',
 'fecha_inicio',
 'calle_hechos',
 'calle_hechos2',
 'colonia_hechos']

In [25]:
#FINAL LIST OF NUMERICAL VARIABLES
LS_NUMVAR = LS_NUMVAR_01 + LS_CATVAREX
LS_NUMVAR

['_id', 'longitud', 'latitud']

In [26]:
# TRANSFORMS CATEGORICAL VARIABLES

df_input = df_raw_01.copy() 

def convertscat(x):
    try:
        y = round(x) #IF THE CATEGORY LOOK LIKE A NUMBER, BETTER ROUND IT
        y = str(y).strip() 
    except:
        try:
            y = str(x).upper().strip() #THE CATEGORY IS TRANSFORMED ONTO UPCASE AND REMOVED THE BLANK TAILS
        except:
            y = np.nan #PUT A NAN IN CASE OF NOT CONVERSION
    return y


for s in LS_CATVAR:
    CATVAR = df_input[s].apply(lambda x: convertscat(x))
    df_input = df_input.drop(columns=[s])
    df_input[s] = CATVAR.astype(str)
    
    print('Variable transformed: ' + str(s))

df_raw_02 = df_input.copy()
del(df_input)

print('\n')
print(df_raw_02.shape)
print('\n')
print(df_raw_02.dtypes)
print('\n')
df_raw_02[LS_CATVAR].head(10)

Variable transformed: ao_hechos
Variable transformed: mes_hechos
Variable transformed: ao_inicio
Variable transformed: mes_inicio
Variable transformed: delito
Variable transformed: fiscalia
Variable transformed: agencia
Variable transformed: unidad_investigacion
Variable transformed: categoria_delito
Variable transformed: alcaldia_hechos
Variable transformed: competencia
Variable transformed: fecha_hechos
Variable transformed: fecha_inicio
Variable transformed: calle_hechos
Variable transformed: calle_hechos2
Variable transformed: colonia_hechos


(1168250, 20)


_id                     object
longitud                object
latitud                 object
tempo                   object
ao_hechos               object
mes_hechos              object
ao_inicio               object
mes_inicio              object
delito                  object
fiscalia                object
agencia                 object
unidad_investigacion    object
categoria_delito        object
alcaldia_hechos         obj

,ao_hechos,mes_hechos,ao_inicio,mes_inicio,delito,fiscalia,agencia,unidad_investigacion,categoria_delito,alcaldia_hechos,competencia,fecha_hechos,fecha_inicio,calle_hechos,calle_hechos2,colonia_hechos
0,2016,ENERO,2016,FEBRERO,DAÃO EN PROPIEDAD AJENA INTENCIONAL,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-2,UI-3CD,DELITO DE BAJO IMPACTO,BENITO JUAREZ,NAN,2016-01-31 22:16:00,2016-02-01T00:25:44,CALZ. DE TLALPAN,LAGO PTE.,AMÃRICAS UNIDAS
1,2016,ENERO,2016,FEBRERO,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,INVESTIGACIÃN PARA LA ATENCIÃN DEL DELITO DE...,ORIENTEII,UI-3CD,ROBO DE VEHÃCULO CON Y SIN VIOLENCIA,IZTAPALAPA,NAN,2016-01-31 20:50:00,2016-02-01T00:52:37,SUR 25 A,GRANJEROS,LOS CIPRESES
2,2016,FEBRERO,2016,FEBRERO,NARCOMENUDEO POSESION SIMPLE,"INVESTIGACIÃN PARA LA ATENCIÃN DE NIÃOS, NI...",57,UI-3CD,DELITO DE BAJO IMPACTO,IZTAPALAPA,NAN,2016-02-01 00:30:00,2016-02-01T01:33:26,BATALLONES ROJOS,NAN,UNIDAD VICENTE GUERRERO
3,2016,ENERO,2016,FEBRERO,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,INVESTIGACIÃN EN IZTAPALAPA,IZP-6,UI-3CD,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,IZTAPALAPA,NAN,2016-01-31 22:00:00,2016-02-01T02:09:11,GERANIO,GAVILLEROS,EMILIANO ZAPATA
4,2015,DICIEMBRE,2016,FEBRERO,DENUNCIA DE HECHOS,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-1,UI-3SD,HECHO NO DELICTIVO,BENITO JUAREZ,NAN,2015-12-25 12:00:00,2016-02-01T02:16:49,SAN FRANCISCO,AVENIDA COLONIA DEL VALLE,DEL VALLE CENTRO
5,2016,FEBRERO,2016,FEBRERO,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,INVESTIGACIÃN EN CUAUHTEMOC,CUH-2,UI-3CD,DELITO DE BAJO IMPACTO,CUAUHTEMOC,NAN,2016-02-01 01:30:00,2016-02-01T02:19:12,AV. CHAPULTEPEC,LIEJA,JUÃREZ
6,2016,ENERO,2016,FEBRERO,LESIONES INTENCIONALES,INVESTIGACIÃN EN TLAHUAC,TLH-2,UI-3SD,DELITO DE BAJO IMPACTO,TLAHUAC,NAN,2016-01-31 22:15:00,2016-02-01T03:15:11,JUAN NEPOMUCENO DE REYES,FELIPE PEÃA PALACIOS,SANTA CECILIA
7,2012,AGOSTO,2016,FEBRERO,FRAUDE,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-1,UI-3SD,DELITO DE BAJO IMPACTO,BENITO JUAREZ,NAN,2012-08-16 12:00:00,2016-02-01T03:44:15,MONTECITO,AV. DE LOS INSURGENTES SUR,NAPOLES
8,2012,DICIEMBRE,2016,FEBRERO,FRAUDE,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-1,UI-3SD,DELITO DE BAJO IMPACTO,BENITO JUAREZ,NAN,2012-12-28 12:00:00,2016-02-01T04:40:59,MONTECITO,NAN,NAPOLES
9,2016,ENERO,2016,FEBRERO,VIOLENCIA FAMILIAR,INVESTIGACIÃN EN CUAUHTEMOC,CUH-8,UI-3SD,DELITO DE BAJO IMPACTO,IZTACALCO,NAN,2016-01-27 11:30:00,2016-02-01T05:57:36,ALVARO OBREGON,NAN,NUEVA SANTA ANITA


In [27]:
#TRANSFORMS NUMERICAL VARIABLES

df_input = df_raw_02.copy() 

def converts(x):
    try:
        y = str(x)                  #FIRST THE VARIABLES IS LOOK AS STRING
        y = y.replace("$", '')      #REMOVES THE $ SIMBOL
        y = y.replace(",", '.')     #Replace the , for . (depends of the numeric format of origin)
        #y = y.replace(",", '')     #Replace the , for '' (depends of the numeric format of origin)
        y = float(y)                # Converts to float
    except:
        y = np.nan
    return y


for s in LS_NUMVAR:
    NUMVAR = df_input[s].apply(lambda x: converts(x))
    df_input = df_input.drop(columns=[s])
    df_input[s] = NUMVAR.astype(float)
    
    
    
    print('Variable transformed: ' + str(s))

df_raw_03 = df_input.copy()
del(df_input)

print('\n')
print(df_raw_03.shape)
print('\n')
print(df_raw_03.dtypes)
print('\n')
df_raw_03[LS_NUMVAR].head(10)

Variable transformed: _id
Variable transformed: longitud
Variable transformed: latitud


(1168250, 20)


tempo                    object
ao_hechos                object
mes_hechos               object
ao_inicio                object
mes_inicio               object
delito                   object
fiscalia                 object
agencia                  object
unidad_investigacion     object
categoria_delito         object
alcaldia_hechos          object
competencia              object
fecha_hechos             object
fecha_inicio             object
calle_hechos             object
calle_hechos2            object
colonia_hechos           object
_id                     float64
longitud                float64
latitud                 float64
dtype: object




,_id,longitud,latitud
0,1.0,-99.140215,19.380315
1,2.0,-99.108403,19.352859
2,3.0,-99.061321,19.356451
3,4.0,-98.976379,19.331633
4,5.0,-99.172189,19.387114
5,6.0,-99.174075,19.421356
6,7.0,-99.001595,19.279864
7,8.0,-99.173480,19.394918
8,9.0,-99.173480,19.394918
9,10.0,-99.126451,19.399273


In [28]:
#VERIFIES THE NUMERICAL CONVERSION 

DF_INPUT = df_raw_03.copy()
DF_INPUT2 = df_raw_02.copy()

for c in LS_NUMVAR:
    print(DF_INPUT[c].value_counts(dropna=False).reset_index().sort_values([c],ascending = False))
    print(DF_INPUT2[c].value_counts(dropna=False).reset_index().sort_values([c],ascending = False))

             index  _id
0        1064129.0    1
778831    131066.0    1
778837    131069.0    1
778836    402192.0    1
778835    131068.0    1
...            ...  ...
389422   1002400.0    1
389423    772128.0    1
389424    131078.0    1
389425    131079.0    1
1168249        2.0    1

[1168250 rows x 2 columns]
           index  _id
0         929792    1
778831    252304    1
778837    317734    1
778836    669664    1
778835    983433    1
...          ...  ...
389422    841329    1
389423   1159793    1
389424     18519    1
389425    499837    1
1168249   160967    1

[1168250 rows x 2 columns]
            index  longitud
0             NaN     46464
1      -99.149570       345
2      -99.070157       277
3      -99.113105       242
4      -99.052963       202
...           ...       ...
316293 -99.176930         1
316294 -99.110325         1
316295 -99.120020         1
316296 -99.074915         1
741014 -99.097498         1

[741015 rows x 2 columns]
                    index  lo

In [29]:
#EXCLUDES REDUNDANT VARIABLES

LS2_SUGVAREXC =[
]

In [30]:
#REMOVE VARIABLE TO EXCLUDE

df_input = df_raw_03.copy()

LS_SUGVAREXC = [x for x in df_input.columns.tolist() if x in LS_SUGVAREXC + LS2_SUGVAREXC]

df_input = df_input.drop(columns=LS_SUGVAREXC)

df_raw_04 = df_input.copy()
del(df_input)

print(df_raw_04.shape)
print(df_raw_04.dtypes)

(1168250, 19)
ao_hechos                object
mes_hechos               object
ao_inicio                object
mes_inicio               object
delito                   object
fiscalia                 object
agencia                  object
unidad_investigacion     object
categoria_delito         object
alcaldia_hechos          object
competencia              object
fecha_hechos             object
fecha_inicio             object
calle_hechos             object
calle_hechos2            object
colonia_hechos           object
_id                     float64
longitud                float64
latitud                 float64
dtype: object


In [31]:
df_input = df_raw_04.copy()

D_DATEVARS = [x for x in df_input.columns if x.find('fecha') != -1]
D_DATEVARS =[x for x in D_DATEVARS if x not in df_informats['index'].tolist()]

df_input[D_DATEVARS]

,fecha_hechos,fecha_inicio
0,2016-01-31 22:16:00,2016-02-01T00:25:44
1,2016-01-31 20:50:00,2016-02-01T00:52:37
2,2016-02-01 00:30:00,2016-02-01T01:33:26
3,2016-01-31 22:00:00,2016-02-01T02:09:11
4,2015-12-25 12:00:00,2016-02-01T02:16:49
...,...,...
1168245,2021-02-27 13:20:00,2021-02-28T22:13:34
1168246,2021-02-27 22:30:00,2021-02-28T22:16:00
1168247,2021-02-28 19:00:00,2021-02-28T22:22:29
1168248,2021-02-28 09:00:00,2021-02-28T22:22:45


In [32]:
#FIXING DATE VARIABLES

df_input = df_raw_04.copy()


def validdate(x,v):
    
    try:
        x1 = x[v][0:10]
    except:
        x1 = None
        
    
    try:
        y1 = datetime.date(int(x1[0:4]),int(x1[5:7]),int(x1[8:10]))
    except:
        y1 = None
    

        
    return y1


for s in D_DATEVARS:
    
    DF_DATE = df_input.apply(lambda x: validdate(x,s),axis=1)
    DF_DATE = pd.DataFrame(pd.to_datetime(DF_DATE, errors='coerce'))
    
    DS_INPUT = df_input.drop(columns=[s])
    df_input[s] = DF_DATE
    
    print('Variable transformed: ' + str(s)) 

    
df_raw_05 = df_input.copy()
del(df_input)

Variable transformed: fecha_hechos
Variable transformed: fecha_inicio


In [33]:
df_raw_05[D_DATEVARS]

,fecha_hechos,fecha_inicio
0,2016-01-31,2016-02-01
1,2016-01-31,2016-02-01
2,2016-02-01,2016-02-01
3,2016-01-31,2016-02-01
4,2015-12-25,2016-02-01
...,...,...
1168245,2021-02-27,2021-02-28
1168246,2021-02-27,2021-02-28
1168247,2021-02-28,2021-02-28
1168248,2021-02-28,2021-02-28


In [34]:
#REPORTS THE FINAL FORMAT FOR THE VARIABLES
df_input = df_raw_05.copy()

DF_FCOLUMNS = df_input.dtypes.reset_index()
DF_FCOLUMNS = DF_FCOLUMNS.rename(columns={'index':'INPUT_VAR',0:'MANDATORY_FORMAT'})

#Final number of numerical and categorical variables

Q_NUMVARS = len(DF_FCOLUMNS[DF_FCOLUMNS['MANDATORY_FORMAT'].astype(str).str.contains('float|int')])
Q_CATARS = len(DF_FCOLUMNS[DF_FCOLUMNS['MANDATORY_FORMAT'].astype(str).str.contains('float|int')==False])
Q_DATEARS = len(DF_FCOLUMNS[DF_FCOLUMNS['MANDATORY_FORMAT'].astype(str).str.contains('date')])

print('Number of Suggested Numerical Variables: ' + str(Q_NUMVARS))   
print('Number of Suggested Categorical Variables: ' + str(Q_CATARS))
print('Number of Final Date Variables: ' + str(Q_DATEARS)) 

Number of Suggested Numerical Variables: 3
Number of Suggested Categorical Variables: 16
Number of Final Date Variables: 2


In [35]:
print(df_raw_05.shape)
print('\n')

i = 0
for c in df_raw_05.columns.tolist():
    i = i +1
    print(i, '-', c)

(1168250, 19)


1 - ao_hechos
2 - mes_hechos
3 - ao_inicio
4 - mes_inicio
5 - delito
6 - fiscalia
7 - agencia
8 - unidad_investigacion
9 - categoria_delito
10 - alcaldia_hechos
11 - competencia
12 - fecha_hechos
13 - fecha_inicio
14 - calle_hechos
15 - calle_hechos2
16 - colonia_hechos
17 - _id
18 - longitud
19 - latitud


In [37]:
# Saving Point

df_raw_05.to_pickle(OUTPUT_DIR + 'df_invest_s1.pkl')